# Interactivity

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from ipywidgets import widgets, Layout
import folium
from folium.plugins import MarkerCluster
from IPython.display import clear_output


# Get and Collapse Data by State, County

In [2]:
# Get cleaned gun violence master dataset and states data
directory = os.path.dirname(str(Path().resolve()) + '\\')
path = os.path.join(directory, 'data', 'final_data.csv')
gunViolence = pd.read_csv(path)

path = os.path.join(directory, 'data', 'county_unemployment.csv')
countyData = pd.read_csv(path)

path = os.path.join(directory, 'data', 'predictions.csv')
predictions = pd.read_csv(path)

In [3]:
predictionsSub = predictions[((predictions['year']==2019) & (predictions['month']==1))]
predictionsSub = predictionsSub.drop(columns=['year', 'month', 'Gun Death Rate (Ranked High to Low)', 'Gun Denials 2008', 'Mental Health 2008', 'Gun Ownership Rank', 'Gun Denials 2017'])

def decode(row):
    for c in predictionsSub.columns:
        if row[c] == 1:
            return c
predictionsSub['State'] = predictionsSub.apply(decode, axis=1)
predictionsSub = predictionsSub[['State', 'predictions']]

In [4]:
stateGeom = pd.read_json('data/us_states_20m.json')
for i in range(len(stateGeom['features'])):
    stateGeom['type'][i] = stateGeom['features'][i]['properties']['NAME']
    stateGeom['features'][i] = stateGeom['features'][i]['geometry']
stateGeom.rename(columns={'type':'State',
                          'features':'Geometry'}, 
                 inplace=True)

In [5]:
stateCols = ['state', 'Mental_Health_Records_Submitted_2008', 'Mental_Health_Records_Submitted_2017', 'Gun_Sale_Denials_2008', 
             'Gun_Sale_Denials_2017', 'Giffords Gun Safety Score',  'Gun Deaths per 100k People (2018)', 
             'Gun Death Rate (Ranked High to Low)', '# of guns per capita', '# of guns registered', 'Permit Type',  
             'Handgun_Carry_Permit_Fee', 'Years_Valid', '5_Year_Cost', 'Happiness Score']
countyCols = ['County', 'Rate']
stateData = gunViolence[stateCols].drop_duplicates('state')
countyData = countyData[countyCols].drop_duplicates('County')
countyData.columns.values[1] = '2015 Unemployment Rate'
countyData = countyData.reset_index()
stateData = stateData.reset_index()
stateData = stateData.drop(columns=['index'])
countyData = countyData[['County', '2015 Unemployment Rate']]
countyData['County'] = countyData['County'].str.replace(' County', '', regex=True)
stateData = stateGeom.merge(stateData, left_on='State', right_on='state')
stateData = stateData.drop(columns=['state'])
stateData = stateData.merge(predictionsSub, left_on='State', right_on='State')

In [6]:
#Rename StateData
stateData.rename(columns={'predictions':'# Predicted Incidents (2019)',
                          '5_Year_Cost':'5-Year Gun Ownership Cost (USD)',
                          'Handgun_Carry_Permit_Fee':'Handgun Carry Permit Fee (USD)',
                          'Years_Valid':'Permit Term (Years)',
                          'Mental_Health_Records_Submitted_2008':'Mental Health Records Submitted (2008)',
                          'Mental_Health_Records_Submitted_2017':'Mental Health Records Submitted (2017)',
                          'Gun_Sale_Denials_2008':'Gun Sale Denials (2008)',
                          'Gun_Sale_Denials_2017':'Gun Sale Denials (2017)'
                         }, 
                 inplace=True)
dropdownOptions = np.concatenate([stateData.columns.values[2:],countyData.columns.values[1:]])

# Display Interactive Map with Data by State (Most Recent Code)

In [7]:
# embed_map function written by user: ocefpaf to patch folium rendering issue with > 80 objects in Google Chrome
def embed_map(m):
    from IPython.display import IFrame
    m.save('index.html')
    return IFrame('index.html', width='100%', height='650px')

# Define list of factors that should be plotted with high values as red instead of green
badFactorsList = ['Gun Deaths per 100k People (2018)', '# of guns per capita', '# of guns registered','2015 Unemployment Rate']
def getColColor(col):
    if (col in badFactorsList):
        return 'OrRd'
    else:
        return 'YlGn'
    
def getMarkerColor(n_killed):
    if (n_killed == 0):
        return '#ffff00'
    elif (n_killed < 2):
        return '#f8d568'
    elif (n_killed < 5):
        return '#ffa500'
    else:
        return '#ff0000'

In [10]:
dropdown = widgets.Dropdown(
    options=dropdownOptions,
    value='5-Year Gun Ownership Cost (USD)',
    description='Factor to Plot:',
    disabled=False,
    layout=Layout(width='40%'),
    style={'description_width': 'initial'}
)

yearSlider = widgets.IntSlider(min=2014,max=2018,step=1,value=2015,description='Year:',layout=Layout(width='40%'),style={'description_width': 'initial'})
numSlider = widgets.IntSlider(min=0,max=20000,step=100,value=500,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
button = widgets.Button(description = "Go!")

display(dropdown)
display(yearSlider)
display(numSlider)
display(button)


def on_button_clicked(b):
    clear_output()
    display(dropdown)
    display(yearSlider)
    display(numSlider)
    display(button)
    folium_map = folium.Map(location=[35, -97], zoom_start=4, tiles="Mapbox Bright")   
    cluster = MarkerCluster(name='Gun Violence Incidents')
    for i, row in gunViolence[gunViolence['year'] == yearSlider.value].sample(frac=1).head(numSlider.value).iterrows():
        cluster.add_child(folium.CircleMarker(location=[row.latitude,row.longitude], radius =5*(row.n_killed+row.n_injured), fill=True, color=getMarkerColor(row.n_killed), popup=folium.Popup('Date: '+str(row.month)+'-'+str(row.day)+'-'+str(row.year)+'\nLives Lost: '+str(row.n_killed),max_width=200)))
    folium_map.add_child(cluster)
    
    if (dropdown.value == '2015 Unemployment Rate'):
        folium.Choropleth(
            geo_data='data/us_counties_20m.json',
            name=dropdown.value,
            data=countyData,
            columns=['County', dropdown.value],
            key_on='feature.properties.NAME',
            fill_color=getColColor(dropdown.value),
            fill_opacity=0.5,
            line_opacity=0.2,
            legend_name=dropdown.value
        ).add_to(folium_map)
    else:
        folium.Choropleth(
            geo_data='data/us_states_20m.json',
            name=dropdown.value,
            data=stateData,
            columns=['State', dropdown.value],
            key_on='feature.properties.NAME',
            fill_color=getColColor(dropdown.value),
            fill_opacity=0.5,
            line_opacity=0.2,
            legend_name=dropdown.value
        ).add_to(folium_map)

    folium.LayerControl().add_to(folium_map)
    toDisp = embed_map(folium_map)
    display(toDisp)

button.on_click(on_button_clicked)

Dropdown(description='Factor to Plot:', index=12, layout=Layout(width='40%'), options=('Mental Health Records Submitted (2008)', 'Mental Health Records Submitted (2017)', 'Gun Sale Denials (2008)', 'Gun Sale Denials (2017)', 'Giffords Gun Safety Score', 'Gun Deaths per 100k People (2018)', 'Gun Death Rate (Ranked High to Low)', '# of guns per capita', '# of guns registered', 'Permit Type', 'Handgun Carry Permit Fee (USD)', 'Permit Term (Years)', '5-Year Gun Ownership Cost (USD)', 'Happiness Score', '# Predicted Incidents (2019)', '2015 Unemployment Rate'), style=DescriptionStyle(description_width='initial'), value='5-Year Gun Ownership Cost (USD)')

IntSlider(value=2015, description='Year:', layout=Layout(width='40%'), max=2018, min=2014, style=SliderStyle(description_width='initial'))

IntSlider(value=500, description='Number of Incidents:', layout=Layout(width='40%'), max=20000, step=100, style=SliderStyle(description_width='initial'))

Button(description='Go!', style=ButtonStyle())

# Subcomponent Demonstration Code

In [8]:
# Testing for Marker Clusters
# Create interactive widgets with mapping function
dropdown = widgets.Dropdown(
    options=stateData.columns.values[1:],
    value='5_Year_Cost',
    description='Factor to Plot:',
    disabled=False,
    layout=Layout(width='40%'),
    style={'description_width': 'initial'}
)

slider = widgets.IntSlider(min=0,max=1000,step=100,value=100,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
button = widgets.Button(description = "Go!")
display(dropdown)
display(slider)
display(button)


def on_button_clicked(b):
    clear_output()
    display(dropdown)
    display(slider)
    display(button)
    folium_map = folium.Map(location=[35, -97], zoom_start=4, tiles="Mapbox Bright")   
    cluster = MarkerCluster()
    for i, row in gunViolence.sample(frac=1).head(slider.value).iterrows():
        cluster.add_child(folium.CircleMarker(location=[row.latitude,  row.longitude], radius = 5*(row.n_killed + row.n_injured), popup = 'Lives Lost: ' + str(row.n_killed)))
    folium_map.add_child(cluster)
    folium.Choropleth(
        geo_data='data/us_states_20m.json',
        name=dropdown.value,
        data=stateData,
        columns=['state', dropdown.value],
        key_on='feature.properties.NAME',
        fill_color=getColor(dropdown.value),
        fill_opacity=0.5,
        line_opacity=0.2,
        legend_name=dropdown.value
    ).add_to(folium_map)
    folium.LayerControl().add_to(folium_map)
    toDisp = embed_map(folium_map)
    display(toDisp)

button.on_click(on_button_clicked)

Dropdown(description='Factor to Plot:', index=12, layout=Layout(width='40%'), options=('Mental_Health_Records_Submitted_2008', 'Mental_Health_Records_Submitted_2017', 'Gun_Sale_Denials_2008', 'Gun_Sale_Denials_2017', 'Giffords Gun Safety Score', 'Gun Deaths per 100k People (2018)', 'Gun Death Rate (Ranked High to Low)', '# of guns per capita', '# of guns registered', 'Permit Type', 'Handgun_Carry_Permit_Fee', 'Years_Valid', '5_Year_Cost', 'Happiness Score'), style=DescriptionStyle(description_width='initial'), value='5_Year_Cost')

IntSlider(value=100, description='Number of Incidents:', layout=Layout(width='40%'), max=1000, step=100, style=SliderStyle(description_width='initial'))

Button(description='Go!', style=ButtonStyle())

In [12]:
# Simple Overlay Demonstration (no Data)

def embed_map(m):
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

folium_map = folium.Map(location=[35, -97],
                        zoom_start=4,
                        tiles="Mapbox Bright")
folium.GeoJson('data/us_counties_20m.json', name='US Counties').add_to(folium_map)
folium.GeoJson('data/us_states_20m.json', name='US States').add_to(folium_map)
folium.LayerControl().add_to(folium_map)
embed_map(folium_map)

In [13]:
# Widget Demonstration Code

button = widgets.Button(description = "Go!")
slider = widgets.IntSlider(min=0,max=5000,step=100,value=100,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
display(slider)
display(button)

def on_button_clicked(b):
    clear_output()
    display(slider)
    display(button)
    folium_map = folium.Map(location=[35, -97],
                            zoom_start=4,
                            tiles="Mapbox Bright")
    for i, row in gunViolence.head(slider.value).iterrows():
        marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], radius = (row['n_killed'] + row['n_injured']))
        marker.add_to(folium_map)
    embed_map(folium_map)
button.on_click(on_button_clicked)

IntSlider(value=2400, description='Number of Incidents:', layout=Layout(width='40%'), max=5000, step=100, styl…

Button(description='Go!', style=ButtonStyle())

In [20]:
folium_map = folium.Map(location=[35, -97],
                        zoom_start=4,
                        tiles="Mapbox Bright")
for i, row in gunViolence.sample(frac=1).head(10000).iterrows():
    marker = folium.CircleMarker(location=[row.latitude,row.longitude], radius =2*(row.n_killed+row.n_injured), fill=True, color=getMarkerColor(row.n_killed), popup=folium.Popup('Date: '+str(row.month)+'-'+str(row.day)+'-'+str(row.year)+'\nLives Lost: '+str(row.n_killed),max_width=200))
    marker.add_to(folium_map)
embed_map(folium_map)